In [2]:
import pandas as pd

In [3]:
pip install apify-client

   ---------------------------------------- 0.0/83.2 kB ? eta -:--:--
   ---------------------------------------- 0.0/83.2 kB ? eta -:--:--
   ---------------------------------------- 0.0/83.2 kB ? eta -:--:--
   ---------------------------------------- 0.0/83.2 kB ? eta -:--:--
   ---------------------------------------- 0.0/83.2 kB ? eta -:--:--
   ---------------------------------------- 0.0/83.2 kB ? eta -:--:--
   ---------------------------------------- 0.0/83.2 kB ? eta -:--:--
   ---------------------------------------- 0.0/83.2 kB ? eta -:--:--
   ---------------------------------------- 0.0/83.2 kB ? eta -:--:--
   ---- ----------------------------------- 10.2/83.2 kB ? eta -:--:--
   ---- ----------------------------------- 10.2/83.2 kB ? eta -:--:--
   ---- ----------------------------------- 10.2/83.2 kB ? eta -:--:--
   ---- ----------------------------------- 10.2/83.2 kB ? eta -:--:--
   ---- ----------------------------------- 10.2/83.2 kB ? eta -:--:--
   ---- -------


[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
input = ["sportscar","car"]

In [ ]:
from apify_client import ApifyClient
from dotenv import load_dotenv
import json
import os

load_dotenv()

# Initialize the ApifyClient with your API token
apikey = os.getenv('API')
client = ApifyClient(apikey)

#Prepare the Actor input
run_input = {
    #"hashtags": ["fyp"],
    "searchQueries": input,
    "resultsPerPage": 10,
    "profileScrapeSections": ["videos"],
    "profileSorting": "latest",
    "excludePinnedPosts": False,
    "searchSection": "",
    "maxProfilesPerQuery": 10,
    "scrapeRelatedVideos": False,
    "shouldDownloadVideos": False,
    "shouldDownloadCovers": False,
    "shouldDownloadSubtitles": False,
    "shouldDownloadSlideshowImages": False,
    "shouldDownloadAvatars": False,
    "shouldDownloadMusicCovers": False,
    "proxyCountryCode": "None",
}

# Run the Actor and wait for it to finish
run = client.actor("GdWCkxBtKWOsKjdch").call(run_input=run_input)

data_items = []
print("Fetching results from dataset...")
for item in client.dataset(run["defaultDatasetId"]).iterate_items():
    data_items.append(item)
print(f"Collected {len(data_items)} items.")

# --- Data Processing Start ---
if data_items:
    # 1. Create initial DataFrame
    df = pd.DataFrame(data_items)

    # 2. Ensure 'authorMeta' values are proper dictionaries. now it's a blob that contain more details of author.
    def parse_json_if_string(data):
        if isinstance(data, str):
            try:
                return json.loads(data)
            except json.JSONDecodeError:
                print(f"Warning: Could not parse authorMeta string: {data[:50]}...")
                return {} # Return an empty dict if parsing fails
        elif isinstance(data, dict):
            return data
        else:
            return {} # Handle other unexpected types

    df['authorMeta'] = df['authorMeta'].apply(parse_json_if_string)

    # 3. Flatten 'authorMeta' into a new DataFrame
    author_meta_df = pd.json_normalize(df['authorMeta'])

    # IMPORTANT: Rename columns in author_meta_df to avoid conflicts with existing columns
    # and to clearly distinguish author-related fields.
    author_meta_df = author_meta_df.add_prefix('author_')

    # 4. Concatenate the original DataFrame (without the nested 'authorMeta' column)
    final_df = pd.concat([df.drop('authorMeta', axis=1), author_meta_df], axis=1)

    # Define the CSV file name
    output_csv_file = "apify_tiktok_combined_data.csv"

    # Save the final DataFrame to a CSV file
    final_df.to_csv(output_csv_file, index=False, encoding='utf-8')
    print(f"Combined data successfully saved to {output_csv_file}")

    print("\nFirst 5 rows of the combined DataFrame:")
    print(final_df.head())
    print("\nColumns in the combined DataFrame:")
    print(final_df.columns.tolist())

else:
    print("No data items were retrieved from the Apify dataset.")



[apify.tiktok-scraper runId:CEy2NnldTWAEoqvrT] -> Status: RUNNING, Message: Starting the crawler.
[apify.tiktok-scraper runId:CEy2NnldTWAEoqvrT] -> Status: RUNNING, Message: Scraped 2/2 search queries
[apify.tiktok-scraper runId:CEy2NnldTWAEoqvrT] -> 2025-07-31T07:09:53.576Z ACTOR: Pulling Docker image of build 7OshLXlRoiesBgRW8 from registry.
[apify.tiktok-scraper runId:CEy2NnldTWAEoqvrT] -> 2025-07-31T07:09:53.578Z ACTOR: Creating Docker container.
[apify.tiktok-scraper runId:CEy2NnldTWAEoqvrT] -> 2025-07-31T07:09:53.723Z ACTOR: Starting Docker container.
[apify.tiktok-scraper runId:CEy2NnldTWAEoqvrT] -> 2025-07-31T07:09:54.054Z Will run command: xvfb-run -a -s "-ac -screen 0 1920x1080x24+32 -nolisten tcp" /bin/sh -c ./start_xvfb_and_run_cmd.sh && npm run start:prod --silent
[apify.tiktok-scraper runId:CEy2NnldTWAEoqvrT] -> 2025-07-31T07:09:55.457Z INFO  System info {"apifyVersion":"3.4.2","apifyClientVersion":"2.12.6","crawleeVersion":"3.13.9","osType":"Linux","nodeVersion":"v20.19.

Fetching results from dataset...
Collected 20 items.
Combined data successfully saved to apify_tiktok_combined_data.csv


In [9]:
final_df.head(5)

,id,text,textLanguage,createTime,createTimeISO,isAd,musicMeta,webVideoUrl,mediaUrls,videoMeta,...,author_bioLink,author_originalAvatarUrl,author_avatar,author_privateAccount,author_following,author_friends,author_fans,author_heart,author_video,author_digg
0,7241962703256866053,Black Supra ❤️‍🔥🛐 | 🎥 HYCADE | #toyota #supra ...,en,1686150840,2023-06-07T15:14:00.000Z,False,"{'musicName': 'original sound', 'musicAuthor':...",https://www.tiktok.com/@editsbyturko/video/724...,[],"{'height': 1024, 'width': 576, 'duration': 15,...",...,None,https://p16-common-sign-va.tiktokcdn-us.com/to...,https://p16-common-sign-va.tiktokcdn-us.com/to...,False,44,0,337700,5700000,215,10700
1,7374858777150967082,Full Vid on “To Be Driven” YT😭 #fyp #cars #tik...,en,1717093120,2024-05-30T18:18:40.000Z,False,"{'musicName': 'original sound', 'musicAuthor':...",https://www.tiktok.com/@tobe.driven/video/7374...,[],"{'height': 1024, 'width': 576, 'duration': 43,...",...,None,https://p19-pu-sign-useast8.tiktokcdn-us.com/t...,https://p19-pu-sign-useast8.tiktokcdn-us.com/t...,False,47,0,65900,3100000,145,69
2,7407242875605503275,🩷🤍 Would you drive this Loveshack Pink + Snow ...,en,1724633132,2024-08-26T00:45:32.000Z,False,"{'musicName': 'original sound', 'musicAuthor':...",https://www.tiktok.com/@vintagebroncos_/video/...,[],"{'height': 1024, 'width': 576, 'duration': 17,...",...,None,https://p19-pu-sign-useast8.tiktokcdn-us.com/t...,https://p19-pu-sign-useast8.tiktokcdn-us.com/t...,False,231,0,644600,21200000,726,2334
3,7533057667275754782,Apex Predator 🏎️\n\n• • •\n\nHighly custom Lam...,en,1753926681,2025-07-31T01:51:21.000Z,False,"{'musicName': 'original sound', 'musicAuthor':...",https://www.tiktok.com/@luxury.speed/video/753...,[],"{'height': 1024, 'width': 576, 'duration': 31,...",...,None,https://p16-sign.tiktokcdn-us.com/tos-useast5-...,https://p16-sign.tiktokcdn-us.com/tos-useast5-...,False,0,0,314300,13600000,1397,342
4,7510606077260582174,If you don’t know about the emergency function...,en,1748699286,2025-05-31T13:48:06.000Z,False,"{'musicName': 'original sound - Driver_Arya_',...",https://www.tiktok.com/@carknowledge_/video/75...,[],"{'height': 1024, 'width': 576, 'duration': 68,...",...,None,https://p19-pu-sign-useast8.tiktokcdn-us.com/t...,https://p19-pu-sign-useast8.tiktokcdn-us.com/t...,False,19,0,3500000,20900000,919,220


In [10]:
#this is the part you get the column u want
desired_columns = [
        'text',              # From the original video data
        'author_id',         # From authorMeta, renamed by add_prefix
        'author_name',
        'author_nickName',
        'author_verified',
        'author_signature',
        'author_fans',
        'author_video',
        'textLanguage'
    ]
df = final_df[desired_columns]

In [15]:
df.head(5)

,text,author_id,author_name,author_nickName,author_verified,author_signature,author_fans,author_video,textLanguage
0,Black Supra ❤️‍🔥🛐 | 🎥 HYCADE | #toyota #supra ...,7051677024888865793,editsbyturko,𝐓𝐔𝐑𝐊𝐎,False,✂️ • 𝓔𝓭𝓲𝓽𝓼 𝓪𝓻𝓮 𝓶𝓲𝓷𝓮 •\n®️ 𝓔𝓭𝓲𝓽𝓼 𝓑𝔂 𝓣𝓾𝓻𝓴𝓸 🇹🇷,337700,215,en
1,Full Vid on “To Be Driven” YT😭 #fyp #cars #tik...,7295130229738832939,tobe.driven,To Be Driven,False,I’m on Youtube too- To Be Driven🏎️,65900,145,en
2,🩷🤍 Would you drive this Loveshack Pink + Snow ...,6896083573389329414,vintagebroncos_,Vintage Modern,False,💬 text 470-729-2853 for inventory ⚙️ the only ...,644600,726,en
3,Apex Predator 🏎️\n\n• • •\n\nHighly custom Lam...,7248746901754348590,luxury.speed,Luxury Speed,False,Content Marketing \nAuto Aesthetics & Car Cult...,314300,1397,en
4,If you don’t know about the emergency function...,7198083580341421099,carknowledge_,Driver_Arya_,False,Follow me to learn more car knowledge！,3500000,919,en


In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   text              20 non-null     object
 1   author_id         20 non-null     object
 2   author_name       20 non-null     object
 3   author_nickName   20 non-null     object
 4   author_verified   20 non-null     bool  
 5   author_signature  20 non-null     object
 6   author_fans       20 non-null     int64 
 7   author_video      20 non-null     int64 
 8   textLanguage      20 non-null     object
dtypes: bool(1), int64(2), object(6)
memory usage: 1.4+ KB


In [ ]:
#if you not using the api coding part just upload yours!
#by go to apify -> tiktokscraper and run the clockwork.
#once u done it go to authors column and download it.
df = pd.read_csv('/content/dataset_tiktok_thaifood.csv')
df.drop(columns=['authorMeta.avatar','authorMeta.privateAccount','authorMeta.ttSeller','authorMeta.bioLink'], inplace=True)

In [ ]:
#!!! IF it doesnt have language column in yours, you can just use these.
#to check if the KOL's signature has the language u want in it.
import re
def contains_thai_characters(text):
    if pd.isna(text):
        return False
    return bool(re.search(r'[\u0E00-\u0E7F]', str(text)))

df['has_thai'] = df['authorMeta.signature'].apply(contains_thai_characters)

df_filtered = df[df['has_thai']].copy()

df_filtered.drop(columns=['has_thai'], inplace=True)

print("\nDataFrame after removing rows where 'authorMeta.signature' has no Thai characters:")
print(df_filtered)


DataFrame after removing rows where 'authorMeta.signature' has no Thai characters:
         authorMeta.name   authorMeta.nickName  authorMeta.verified  \
0         praneatseafood    ร้านอาหารประณีต 🦀✨                False   
1     nourivebliss.ncafe  Nourive Bliss N.Cafe                False   
2          krua.pa.pleon          ครัวพาเพลิน🍳                False   
4         annhomecooking        Annhomecooking                False   
5                bee_nil            BB’s World                False   
...                  ...                   ...                  ...   
1344        arm_watunyou        อาร์มหรอยอีหลี                False   
1345        arm_watunyou        อาร์มหรอยอีหลี                False   
1346        arm_watunyou        อาร์มหรอยอีหลี                False   
1347        arm_watunyou        อาร์มหรอยอีหลี                False   
1348        arm_watunyou        อาร์มหรอยอีหลี                False   

                                   authorMeta.signature  author

In [18]:
#Apply Language Filter
desired_language = "en"
if desired_language: # Check if the user actually provided a language
    print(f"Filtering for text language: '{desired_language}'")
    df_filtered = df[
        df['textLanguage'].str.lower() == desired_language
    ]
    print(f"After filtering, {len(df)} rows remain.")
else:
    print("No language filter applied. Keeping all languages.")

Filtering for text language: 'en'
After filtering, 15 rows remain.


In [20]:
df_filtered.head(6)

,text,author_id,author_name,author_nickName,author_verified,author_signature,author_fans,author_video,textLanguage
0,Black Supra ❤️‍🔥🛐 | 🎥 HYCADE | #toyota #supra ...,7051677024888865793,editsbyturko,𝐓𝐔𝐑𝐊𝐎,False,✂️ • 𝓔𝓭𝓲𝓽𝓼 𝓪𝓻𝓮 𝓶𝓲𝓷𝓮 •\n®️ 𝓔𝓭𝓲𝓽𝓼 𝓑𝔂 𝓣𝓾𝓻𝓴𝓸 🇹🇷,337700,215,en
1,Full Vid on “To Be Driven” YT😭 #fyp #cars #tik...,7295130229738832939,tobe.driven,To Be Driven,False,I’m on Youtube too- To Be Driven🏎️,65900,145,en
2,🩷🤍 Would you drive this Loveshack Pink + Snow ...,6896083573389329414,vintagebroncos_,Vintage Modern,False,💬 text 470-729-2853 for inventory ⚙️ the only ...,644600,726,en
3,Apex Predator 🏎️\n\n• • •\n\nHighly custom Lam...,7248746901754348590,luxury.speed,Luxury Speed,False,Content Marketing \nAuto Aesthetics & Car Cult...,314300,1397,en
4,If you don’t know about the emergency function...,7198083580341421099,carknowledge_,Driver_Arya_,False,Follow me to learn more car knowledge！,3500000,919,en
6,Pink car wrap for the new Scat Pack 🔥✨\n\nI al...,6792750260454425601,jaythewrapspecialist,Jay The Wrap Specialist,False,"📍Houston, TX 🇺🇸🇸🇻\n📧 Promo@wrapleaders.com\n👇W...",2300000,453,en


In [24]:
dfsortbyfans = df_filtered.sort_values(by='author_fans',ascending=False)
dfsortbyfans.head(7)

,text,author_id,author_name,author_nickName,author_verified,author_signature,author_fans,author_video,textLanguage
16,This is the glass floor car from the future 🤯🔥...,6790002010382812165,supercarblondie,Supercar Blondie,True,Auctions for the world's rarest cars @SBX Cars...,20200000,1199,en
4,If you don’t know about the emergency function...,7198083580341421099,carknowledge_,Driver_Arya_,False,Follow me to learn more car knowledge！,3500000,919,en
6,Pink car wrap for the new Scat Pack 🔥✨\n\nI al...,6792750260454425601,jaythewrapspecialist,Jay The Wrap Specialist,False,"📍Houston, TX 🇺🇸🇸🇻\n📧 Promo@wrapleaders.com\n👇W...",2300000,453,en
14,I've never put my hands on a Nissan GT-R T-Spe...,6941506180595893254,georgejsaliba,George Saliba,False,Vehicle Acquisition Pro\nBuy+Sell | Ewing NJ\n...,1100000,1636,en
2,🩷🤍 Would you drive this Loveshack Pink + Snow ...,6896083573389329414,vintagebroncos_,Vintage Modern,False,💬 text 470-729-2853 for inventory ⚙️ the only ...,644600,726,en
0,Black Supra ❤️‍🔥🛐 | 🎥 HYCADE | #toyota #supra ...,7051677024888865793,editsbyturko,𝐓𝐔𝐑𝐊𝐎,False,✂️ • 𝓔𝓭𝓲𝓽𝓼 𝓪𝓻𝓮 𝓶𝓲𝓷𝓮 •\n®️ 𝓔𝓭𝓲𝓽𝓼 𝓑𝔂 𝓣𝓾𝓻𝓴𝓸 🇹🇷,337700,215,en
3,Apex Predator 🏎️\n\n• • •\n\nHighly custom Lam...,7248746901754348590,luxury.speed,Luxury Speed,False,Content Marketing \nAuto Aesthetics & Car Cult...,314300,1397,en


In [ ]:
dfsortbyfansunique = dfsortbyfans.drop_duplicates(subset=['authorMeta.name'])

In [ ]:
dfsortbyfansunique.head(20)

,authorMeta.name,authorMeta.nickName,authorMeta.verified,authorMeta.signature,authorMeta.fans,authorMeta.video,authorMeta.id,text
128,yeeyeehtay0113,เจ้าหญิงกบ❤️,False,🙏❤️ครอบครัวสายฮา💝🙏\n ❤️รับงานรีวิวค่ะ: LINE👉...,9000000,3034,6864526597882463233,อร่อย​มาก​ค่ะ​❤️
100,bella_bell22,เบลล่า วิถีไทย,False,ติดต่องาน Line ID : bellsy\n📍เล่นช่องนี้ช่องเด...,7200000,1314,6528496150231695362,โจ๊กมาม่า #เบลล่าวิถีไทย #นักชิมอาหาร #11x11Fo...
54,kinkubky,กินกับกี้,False,Youtubeช่อง กินกับกี้\nติดต่องาน Line :@thelem...,5100000,2262,6809176177791632385,#ฉ่ํามั๊ยละ #กิน #ตํากุ้งสด #อร่อย #แซ่บ #tikt...
556,icezy159,ไอซ์ซี่ กินยั่ว,False,ติดต่องาน lineid : icezy168\nเพจ ไอซ์ซี่ กินยั...,5000000,1684,6519602653701783554,แกงเห็ดเผาะ #ยั่วๆจร้า #นักชิมอาหาร #อร่อยบอกต...
151,achasai,อาลี่ก๋ง. ( 4.6 M ) 🪞👸💗🌷✨,False,For work // line : @952eshvp\nInstagram: aach...,4600000,3424,75635744318,งู๊ยยยย ไข่ดองและแซลมอนดอง 🤤 อร่อยมากกก ฟิน 🫶🏻...
55,best_babeee,เบสเบบี๋ 👶🏻,False,FB: Apicha Insuwan \nIG : best_apicha \nติดต่อ...,3500000,4637,73975939911,ทานเล่นเบาๆกับแม่ยัยบี๋ งานวัดไร่ขิงปี68 🍨🧀🍗🍹🍫...
41,alumilightkitchen7,พ่อบ้านสดใสแม่บ้านหดหู่,False,ขอบคุณทุกคนที่ติดตาม🙏🥰\nID:suningdameinv\nเพจ:...,3200000,1157,6564513206260334593,เมื่อทายาทอยากกินต้องได้กิน #ครัวอลูมีไล้ #พ่อ...
887,tiktokshop_th,TikTokShop_TH,True,TikTok Shop ช้อปสนุกทุกตะกร้า🛒,3100000,1127,7095929463933912070,เมื่อคุณลืมพกความมั่นใจมาจากบ้าน บู้สความมั่น...
95,mastertomtomxoxo,MasterTomTom,False,🎩\nwork- @mastertomtom (มี@)\nงดนำคลิปไปใช้ต่อ...,3100000,274,7014785101205423130,ทำ‘ส้มตำยายเพลิงนรกร้านดัง กรอบแซ่บระเบิดล้นคร...
99,hector.book,แล้วแต่แฟนเลยครับ,False,Book & Air แฟนผมกินเก่ง 😂\nเจ้าของแบรนด์ lumiskin,3000000,424,78239935037,ตอบกลับ @dm.fs คุณแม่แฟนยึดช่องผมแล้วครับ เก่ง...


In [ ]:
pip install pythainlp

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.3/19.3 MB 82.8 MB/s eta 0:00:00


In [ ]:
from pythainlp.tokenize import word_tokenize
from pythainlp.corpus import thai_stopwords

# Load Thai stopwords once
thai_stop_words = set(thai_stopwords())

def preprocess_text(text):
    if not isinstance(text, str): # Handle non-string inputs like None
        return ""

    emoji_pattern = re.compile(
        "["
        "\U0001F600-\U0001F64F"  # emoticons
        "\U0001F300-\U0001F5FF"  # symbols & pictographs
        "\U0001F680-\U0001F6FF"  # transport & map symbols
        "\U0001F1E0-\U0001F1FF"  # flags (iOS)
        "\U00002702-\U000027B0"  # Dingbats
        "\U000024C2-\U0001F251"
        "\U0001F900-\U0001F9FF"  # Supplemental Symbols and Pictographs
        "\U00002600-\U000026FF"  # Miscellaneous Symbols
        "\U00002500-\U000025FF"  # Box Drawing, Block Elements
        "]+", flags=re.UNICODE
    )
    text = emoji_pattern.sub(r'', text)
    # Remove characters that are NOT Thai letters, numbers, #, @, or spaces
    text = re.sub(r'[^\u0E00-\u0E7F\s]', '', text)

    tokens = word_tokenize(text, keep_whitespace=False)


    tokens = [word for word in tokens if word not in thai_stop_words and word.strip() != '']

    return " ".join(tokens)


dfsortbyfansunique['processed_signature'] = dfsortbyfansunique['authorMeta.signature'].apply(preprocess_text)
dfsortbyfansunique['processed_text'] = dfsortbyfansunique['text'].apply(preprocess_text)

dfsortbyfansunique['influencer_combined_text'] = dfsortbyfansunique['processed_signature'] + " " + dfsortbyfansunique['processed_text']

brand_description_text = (
    "โค้ก กินกับอะไรก็อร่อย อาหารเผ็ดๆยิ่งต้องกินกับโค้กเลย "
)

processed_brand_text = preprocess_text(brand_description_text)

print("Processed Brand Text:")
print(processed_brand_text)
print("\nProcessed Influencer Texts (first 2 rows):")
print(dfsortbyfansunique[['authorMeta.name', 'influencer_combined_text']].head(2).to_string())

Processed Brand Text:
โค้ก กิน อร่อย อาหาร เผ็ด กิน โค้ก

Processed Influencer Texts (first 2 rows):
    authorMeta.name                                                                                  influencer_combined_text
128  yeeyeehtay0113                                                                        ครอบครัว สาย ฮา รับงาน รีวิว อร่อย
100    bella_bell22  ติดต่อ งาน เล่น ช่อง ช่อง ห้าม คลิป แจ้ง โจ๊ก มาม่า เบล ล่า วิถี ไทย ชิม อาหาร โรบินฮู้ด กิน เชฟ มือใหม่


/tmp/ipython-input-16-271021785.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfsortbyfansunique['processed_signature'] = dfsortbyfansunique['authorMeta.signature'].apply(preprocess_text)
/tmp/ipython-input-16-271021785.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfsortbyfansunique['processed_text'] = dfsortbyfansunique['text'].apply(preprocess_text)
/tmp/ipython-input-16-271021785.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .l

In [ ]:
print(dfsortbyfansunique['influencer_combined_text'].head())
print("\n"+processed_brand_text)

128                   ครอบครัว สาย ฮา รับงาน รีวิว อร่อย
100    ติดต่อ งาน เล่น ช่อง ช่อง ห้าม คลิป แจ้ง โจ๊ก ...
54     ช่อง กิน กี้ ติดต่อ งาน ยีน ฉ่ํามั๊ย กิน ตํา ก...
556    ติดต่อ งาน เพจ ไอซ์ ซี่ กิน ยั่ว แฟน เพจ เฟส ส...
151    อนุญาต ดูด คลิป กรณี งู๊ยยยย ไข่ ดอง แซลมอน ดอ...
Name: influencer_combined_text, dtype: object

โค้ก กิน อร่อย อาหาร เผ็ด กิน โค้ก


In [40]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Create a corpus of all texts (brand + all influencer texts)
corpus = [processed_brand_text] + dfsortbyfansunique['influencer_combined_text'].tolist()

# max_features can limit the vocabulary size to improve performance/focus on most important words
vectorizer = TfidfVectorizer(max_features=4000)

# Fit the vectorizer on the entire corpus and transform
tfidf_matrix = vectorizer.fit_transform(corpus)

# Separate the brand vector from influencer vectors
brand_vector = tfidf_matrix[0:1]
influencer_vectors = tfidf_matrix[1:]


In [ ]:
similarity_scores = cosine_similarity(brand_vector, influencer_vectors).flatten()

dfsortbyfansunique['relevance_score'] = similarity_scores

print("\nInfluencers with Relevance Scores:")
print(dfsortbyfansunique[['authorMeta.name', 'authorMeta.fans', 'relevance_score']].sort_values(by='relevance_score', ascending=False))


Influencers with Relevance Scores:
               authorMeta.name  authorMeta.fans  relevance_score
415             nurse.enjoyeat           324500         0.273034
188                  shibacook          2100000         0.250146
412                    j100.kg          1100000         0.247859
390                   faymily_            89200         0.236992
433   nutthanon_loetphatphicha          2600000         0.177942
...                        ...              ...              ...
959     ruennoppagaorestaurant              792         0.000000
339                  tiltok_90              211         0.000000
521                cainnnn_nnn              712         0.000000
1106           tiktok_ajarn.wi              177         0.000000
479                   timj0004               15         0.000000

[171 rows x 3 columns]


/tmp/ipython-input-37-470239746.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfsortbyfansunique['relevance_score'] = similarity_scores


In [42]:
weight_relevance = 0.6
weight_fans = 0.1

# Calculate the total score
max_fans = dfsortbyfansunique['authorMeta.fans'].max()
dfsortbyfansunique['normalized_fans'] = dfsortbyfansunique['authorMeta.fans'] / max_fans # Scale fans to 0-1

dfsortbyfansunique['total_score'] = (dfsortbyfansunique['relevance_score'] * weight_relevance) + \
                                (dfsortbyfansunique['normalized_fans'] * weight_fans)

# Sort by the total score to get your final ranked list
final_ranked_influencers = dfsortbyfansunique.sort_values(by='total_score', ascending=False)

print("\nFinal Ranked Influencers (by Total Score):")
print(final_ranked_influencers[['authorMeta.name', 'authorMeta.fans', 'relevance_score', 'total_score']].head(10))


Final Ranked Influencers (by Total Score):
              authorMeta.name  authorMeta.fans  relevance_score  total_score
188                 shibacook          2100000         0.250146     0.173421
415            nurse.enjoyeat           324500         0.273034     0.167426
412                   j100.kg          1100000         0.247859     0.160938
390                  faymily_            89200         0.236992     0.143186
433  nutthanon_loetphatphicha          2600000         0.177942     0.135654
128            yeeyeehtay0113          9000000         0.048802     0.129281
121                 mawin_twp          2900000         0.111934     0.099383
142              sukanyapakin          1000000         0.145463     0.098389
124              dingdongeiei           246300         0.157419     0.097188
54                   kinkubky          5100000         0.067425     0.097121


/tmp/ipython-input-42-922140075.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfsortbyfansunique['normalized_fans'] = dfsortbyfansunique['authorMeta.fans'] / max_fans # Scale fans to 0-1
/tmp/ipython-input-42-922140075.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfsortbyfansunique['total_score'] = (dfsortbyfansunique['relevance_score'] * weight_relevance) + \
